In [12]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

# Set the path to your Google Cloud Vision API JSON key file
credentials_path = '/home/goutam/Downloads/clever-courier-412909-a58ab09d7051.json'


class TextExtractorApp:
    def __init__(self, root):
        root.geometry("500x500")
        self.root = root
        self.root.title("Text Extractor App")

        # Create widgets
        self.label = tk.Label(
            root, text="Choose an image:", font=('Arial', 15))
        self.label.pack(padx=100, pady=10)

        self.choose_button = tk.Button(self.root, text="Choose Image", font=(
            'Arial', 15), command=self.choose_image)
        self.choose_button.pack(pady=20)

        self.image_label = tk.Label(root)
        self.image_label.pack()

        self.result_label = tk.Label(root, text="Processed Text:")
        self.result_label.pack(pady=10)

        self.result_text = tk.Text(root, height=40, width=40)
        self.result_text.pack(pady=10)

        # Initialize Google Cloud Vision API client
        self.client = vision_v1.ImageAnnotatorClient.from_service_account_file(
            credentials_path)

    
    def choose_image(self):
        if not self.client:
            messagebox.showerror("Error", "Google Cloud Vision client not initialized.")
            return
        
        try:
            file_path = filedialog.askopenfilename(
                initialdir="/",  
                title="Select an image file",
                filetypes=[("Image files", "png;jpg;jpeg"), ("All files", "*.*")])
            
            # Debug print to check if file_path is correctly retrieved
            print(f"Selected file: {file_path}")

            if file_path:
                self.display_image(file_path)
                text = self.extract_text(file_path)
                
                # Debug print to check the extracted text
                print(f"Extracted text: {text}")
    
                self.result_text.delete(1.0, tk.END)
                self.result_text.insert(tk.END, text)
            else:
                messagebox.showinfo("Info", "No image selected.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")


    
    # def choose_image(self):
    #     file_path = filedialog.askopenfilename(
    #         filetypes=[("Image files", "*.png;*.jpg;")])
    #     if file_path:
    #         self.display_image(file_path)
    #         text = self.extract_text(file_path)
    #         self.result_text.delete(1.0, tk.END)
    #         self.result_text.insert(tk.END, text)



    
    def display_image(self, file_path):
        image = Image.open(file_path)
        image = image.resize((250, 250))  # Resize for better display
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def extract_text(self, file_path):
        with open(file_path, 'rb') as image_file:
            content = image_file.read()

        image = types.Image(content=content)
        response = self.client.text_detection(image=image)
        texts = response.text_annotations

        if texts:
            return texts[0].description
        else:
            return "No text found"


if __name__ == "__main__":
    root = tk.Tk()
    app = TextExtractorApp(root)
    root.mainloop()

Selected file: ()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/tmp/ipykernel_40673/990124623.py", line 64, in choose_image
    messagebox.showinfo("Info", "No image selected.")
    ^^^^^^^^^^
NameError: name 'messagebox' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/goutam/miniconda3/envs/test/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_40673/990124623.py", line 66, in choose_image
    messagebox.showerror("Error", f"An error occurred: {e}")
    ^^^^^^^^^^
NameError: name 'messagebox' is not defined
